# Load Back Projected ICA Data

In [1]:
import scipy.io as sio

In [10]:
filepath = '/Users/apple/Desktop/eeglab14_1_2b/ICA_Data/EEGData1.mat'
mat_contents = sio.loadmat(filepath)
ica = mat_contents['data']
temp_trial = ica[:,:,1]
trial = ica.shape[2]
print("{}: {}".format("Total number of trials is", trial))
channel = temp_trial.shape[0]
print("{}: {}".format("Total number of channels in each trial is", channel))
timepoint = temp_trial.shape[1]
print("{}: {}".format("Total number of time points in per channel per trial is", timepoint))

Total number of trials is: 40
Total number of channels in each trial is: 32
Total number of time points in per channel per trial is: 8064


# Testing between 1 trial 2 channels with GC

In [13]:
import statsmodels.tsa.stattools as stm
import numpy as np

In [14]:
#we just gonna pick temp_trial
print(temp_trial.shape)

(32, 8064)


In [36]:
a = np.asarray(temp_trial[0,:])
b = np.asarray(temp_trial[1,:])
x = np.vstack((a, b)).T
print(x.shape)

(8064, 2)


In [86]:
from statsmodels.tsa.ar_model import AR
model = AR(a)
model_fit = model.fit()
print('Lag: %s' % model_fit.k_ar)
maxlag = model_fit.k_ar #dimension.fnn(x[0])

Lag: 36


In [87]:
addconst = True
verbose = True

In [88]:
result = stm.grangercausalitytests(x, maxlag, addconst, verbose)
optimal_lag = -1
F_test = -1.0
for key in result.keys():
    _F_test_ = result[key][0]['params_ftest'][0]
    if _F_test_ > F_test:
        F_test = _F_test_
        optimal_lag = key


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=137.4970, p=0.0000  , df_denom=8060, df_num=1
ssr based chi2 test:   chi2=137.5481, p=0.0000  , df=1
likelihood ratio test: chi2=136.3881, p=0.0000  , df=1
parameter F test:         F=137.4970, p=0.0000  , df_denom=8060, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=311.0022, p=0.0000  , df_denom=8057, df_num=2
ssr based chi2 test:   chi2=622.3905, p=0.0000  , df=2
likelihood ratio test: chi2=599.5351, p=0.0000  , df=2
parameter F test:         F=311.0022, p=0.0000  , df_denom=8057, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=283.9087, p=0.0000  , df_denom=8054, df_num=3
ssr based chi2 test:   chi2=852.4664, p=0.0000  , df=3
likelihood ratio test: chi2=810.3369, p=0.0000  , df=3
parameter F test:         F=283.9087, p=0.0000  , df_denom=8054, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=215.8010, p=0.

ssr based F test:         F=27.2416 , p=0.0000  , df_denom=7979, df_num=28
ssr based chi2 test:   chi2=768.2134, p=0.0000  , df=28
likelihood ratio test: chi2=733.6784, p=0.0000  , df=28
parameter F test:         F=27.2416 , p=0.0000  , df_denom=7979, df_num=28

Granger Causality
number of lags (no zero) 29
ssr based F test:         F=25.3977 , p=0.0000  , df_denom=7976, df_num=29
ssr based chi2 test:   chi2=741.9811, p=0.0000  , df=29
likelihood ratio test: chi2=709.6955, p=0.0000  , df=29
parameter F test:         F=25.3977 , p=0.0000  , df_denom=7976, df_num=29

Granger Causality
number of lags (no zero) 30
ssr based F test:         F=25.2617 , p=0.0000  , df_denom=7973, df_num=30
ssr based chi2 test:   chi2=763.6484, p=0.0000  , df=30
likelihood ratio test: chi2=729.5026, p=0.0000  , df=30
parameter F test:         F=25.2616 , p=0.0000  , df_denom=7973, df_num=30

Granger Causality
number of lags (no zero) 31
ssr based F test:         F=23.9798 , p=0.0000  , df_denom=7970, df_num=3

/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 35, but rank is 33
  'rank is %d' % (J, J_), ValueWarning)
/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 36, but rank is 30
  'rank is %d' % (J, J_), ValueWarning)


In [89]:
print("{} {}".format("We are going to look into the GC with Optimal Lag of", optimal_lag))

We are going to look into the GC with Optimal Lag of 2


We consider the p-value of the test as a measure for Granger causality: rejection of ℋ0 (p < 0.03) signifies Granger causality, acceptance means non-causality.

The causality relations drawn from systems with very small values of |det(ΛˆI)| are not meaningful

In [92]:
if (result[optimal_lag][0]['params_ftest'][1] < 0.03 and AR):
    print(result[optimal_lag][0]['params_ftest'][0])


311.00222869083973


# Suppress Printing

In [106]:
import sys, os
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

def enablePrint():
    sys.stdout = sys.__stdout__

# Compute one Multivariant Granger Causality Matrix MGCM

In [107]:
from matplotlib import pyplot
import time

In [ ]:
time_start = time.clock()
MGCM = np.zeros((channel,channel))
for i in range(channel):
    for j in range(channel):
        if i == j:
            enablePrint()
            print("{}:{}".format(i,j))
            MGCM[i,j] = 0
        blockPrint()
        a = np.asarray(temp_trial[i,:])
        b = np.asarray(temp_trial[j,:])
        x = np.vstack((a, b)).T
        model = AR(a)
        model_fit = model.fit()
        maxlag = model_fit.k_ar
        if maxlag > 5:
            maxlag = 5
        result = stm.grangercausalitytests(x, maxlag, addconst = True, verbose = True)
        optimal_lag = -1
        F_test = -1.0
        for key in result.keys():
            _F_test_ = result[key][0]['params_ftest'][0]
            if _F_test_ > F_test:
                F_test = _F_test_
                optimal_lag = key
        if (result[optimal_lag][0]['params_ftest'][1] < 0.03 and AR):
            MGCM[i,j] = result[optimal_lag][0]['params_ftest'][0]
        else:
            MGCM[i,j] = 0
diag = np.max(MGCM)
for i in range(channel):
    for j in range(channel):
        if i == j:
            MGCM[i,j] = diag
        else:
            MGCM[i,j] = MGCM[i,j]/diag
            
pyplot.matshow(MGCM)
pyplot.show()
time_elapsed = (time.clock() - time_start)
print("{}: {}".format("Time Used", time_elapsed))